In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pylab as pl
import subprocess as sp
import astropy.io.fits as pyfits
import pandas as pd

import om10_lensing_equations as ole

In [15]:
def load_in_data_sne():

    sne_host_bulge = pd.read_csv("./data/sne_host_bulge.txt")
    sne_host_disk = pd.read_csv("./data/sne_host_disk.txt")
    
    idx = sne_host_bulge['image_number'] == 0
    shb_purged = sne_host_bulge[:][idx]
    shd_purged = sne_host_disk[:][idx]

    return shb_purged, shd_purged


def create_cats_sne(index, hdu_list, ahb_list, _list):
    
    twinkles_ID = ahb_list['twinkles_system'][index]
    
    print twinkles_ID
    
    print hdu_list[1].data['twinklesId']
    
    idx = hdu_list[1].data['twinklesId'] == twinkles_ID
    lid = hdu_list[1].data['LENSID'][idx][0]
    xl1 = 0.0
    xl2 = 0.0
    vd = hdu_list[1].data['VELDISP'][idx][0]   # needed from OM10
    zd = hdu_list[1].data['ZLENS'][idx][0]
    ql  = 1.0 - hdu_list[1].data['ELLIP'][idx][0]
    phi= hdu_list[1].data['PHIE'][idx][0]

    ys1 = ahb_list['x'][index]
    ys2 = hdu_list['y'][index]

    ext_shr = hdu_list[1].data['GAMMA'][idx][0]
    ext_phi = hdu_list[1].data['PHIG'][idx][0]

    ximg = hdu_list[1].data['XIMG'][idx][0]
    yimg = hdu_list[1].data['YIMG'][idx][0]

    #----------------------------------------------------------------------------
    lens_cat = {'xl1'        : xl1,
                'xl2'        : xl2,
                'ql'         : ql,
                'vd'         : vd,
                'phl'        : phi,
                'gamma'      : ext_shr,
                'phg'        : ext_phi,
                'zl'         : zd,
                'ximg'       : ximg,
                'yimg'       : yimg,
                'twinklesid' : twinkles_ID,
                'lensid'     : lid,
                'index'      : index}
    
    #----------------------------------------------------------------------------
    mag_src_b = ahb_list['phosimMorm'][index]
    qs_b = ahb_list['minorAxis'][index]/ahb_list['majorAxis'][index]
    Reff_src_b = np.sqrt(ahb_list['minorAxis'][index]*ahb_list['majorAxis'][index])
    phs_b = ahb_list['positionAngle'][index]
    ns_b = ahb_list['sindex'][index]
    zs_b = ahb_list['redshift'][index]
    sed_src_b = ahb_list['sedFilepath'][index]
    
    srcsP_bulge = {'ys1'          : ys1,
                   'ys2'          : ys2,
                   'mag_src'      : mag_src_b,
                   'Reff_src'     : Reff_src_b,
                   'qs'           : qs_b,
                   'phs'          : phs_b,
                   'ns'           : ns_b,
                   'zs'           : zs_b,
                   'sed_src'      : sed_src_b,                         
                   'components'   : 'bulge'}
    
    #----------------------------------------------------------------------------
    mag_src_d = _list['phosimMorm'][index]
    qs_d = _list['minorAxis'][index]/_list['majorAxis'][index]
    Reff_src_d = np.sqrt(_list['minorAxis'][index]*_list['majorAxis'][index])
    phs_d = _list['positionAngle'][index]
    ns_d = _list['sindex'][index]
    zs_d = _list['redshift'][index]
    sed_src_d = _list['sedFilepath'][index]

    srcsP_disk = {'ys1'          : ys1,
                  'ys2'          : ys2,
                  'mag_src'      : mag_src_d,
                  'Reff_src'     : Reff_src_d,
                  'qs'           : qs_d,
                  'phs'          : phs_d,
                  'ns'           : ns_d,
                  'zs'           : zs_d,
                  'sed_src'      : sed_src_d,
                  'components'   : 'disk'}
    
    #----------------------------------------------------------------------------

    return lens_cat, srcsP_bulge, srcsP_disk


def lensed_sersic_2d(xi1, xi2, yi1, yi2, source_cat, lens_cat):
    #----------------------------------------------------------------------
    ysc1     = source_cat['ys1']        # x position of the source, arcseconds
    ysc2     = source_cat['ys2']        # y position of the source, arcseconds
    mag_tot  = source_cat['mag_src']    # total mitude of the source
    Reff_arc = source_cat['Reff_src']   # Effective Radius of the source, arcseconds
    qs       = source_cat['qs']         # axis ratio of the source, b/a
    phs      = source_cat['phs']        # orientation of the source, degree
    ndex     = source_cat['ns']         # index of the source

    #----------------------------------------------------------------------
    g_limage = ole.sersic_2d(yi1,yi2,ysc1,ysc2,Reff_arc,qs,phs,ndex)
    g_source = ole.sersic_2d(xi1,xi2,ysc1,ysc2,Reff_arc,qs,phs,ndex)

    mag_lensed = mag_tot - 2.5*np.log(np.sum(g_limage)/np.sum(g_source))

    return mag_lensed, g_limage


def generate_lensed_host(xi1, xi2, lens_P, srcP_b, srcP_d):
    dsx = 0.01
    xlc1 = lens_P['xl1']              # x position of the lens, arcseconds
    xlc2 = lens_P['xl2']              # y position of the lens, arcseconds
    rlc  = 0.0                          # core size of Non-singular Isothermal Ellipsoid
    vd   = lens_P['vd']               # velocity dispersion of the lens
    zl   = lens_P['zl']               # redshift of the lens
    zs   = srcP_b['zs']             # redshift of the source
    rle  = ole.re_sv(vd, zl, zs)        # Einstein radius of lens, arcseconds.
    ql   = lens_P['ql']               # axis ratio b/a
    le   = ole.e2le(1.0 - ql)           # scale factor due to projection of ellpsoid
    phl  = lens_P['phl']              # position angle of the lens, degree
    eshr = lens_P['gamma']            # external shear
    eang = lens_P['phg']              # position angle of external shear
    ekpa = 0.0                          # external convergence

    #----------------------------------------------------------------------
    ai1, ai2 = ole.alphas_sie(xlc1, xlc2, phl, ql, rle, le,
                              eshr, eang, ekpa, xi1, xi2)

    yi1 = xi1 - ai1
    yi2 = xi2 - ai2
    #----------------------------------------------------------------------------

    lensed_mag_b, lensed_image_b = lensed_sersic_2d(xi1,xi2,yi1,yi2,srcP_b,lens_P)

    fits_limg_b = "./outputs/lensed_bulges/" + str(lens_P['index']) \
              + "_" + str(srcP_b['components']) + "_" + str(srcP_b['sed_src'].split('/')[0]) \
              + "_" + str(srcP_d['sed_src'].split('/')[1]) \
              + "_" + str(srcP_b['zs']) + "_" + str(srcP_b['mag_src'])\
              + "_" + str(lensed_mag_b) + "_" + str(dsx) + ".fits"

    pyfits.writeto(fits_limg_b, lensed_image_b.astype("float32"), overwrite=True)

    cmd_b = "bzip2 -f " + fits_limg_b
    sp.call(cmd_b, shell=True)
    
    pl.figure(figsize=(8,8))
    pl.contourf(xi1,xi2,lensed_image_b)
    pl.plot(lens_P['ximg'][np.nonzero(lens_P['ximg'])], lens_P['yimg'][np.nonzero(lens_P['yimg'])], 'bx')
    #----------------------------------------------------------------------------

    lensed_mag_d, lensed_image_d = lensed_sersic_2d(xi1,xi2,yi1,yi2,srcP_d,lens_P)

    fits_limg_d = "./outputs/lensed_disks/" + str(lens_P['index']) \
              + "_" + str(srcP_d['components']) + "_" + str(srcP_d['sed_src'].split('/')[0]) \
              + "_" + str(srcP_d['sed_src'].split('/')[1]) \
              + "_" + str(srcP_d['zs']) + "_" + str(srcP_d['mag_src']) \
              + "_" + str(lensed_mag_d) + "_" + str(dsx) + ".fits"

    pyfits.writeto(fits_limg_d, lensed_image_d.astype("float32"), overwrite=True)

    cmd_d = "bzip2 -f " + fits_limg_d
    sp.call(cmd_d, shell=True)
    
    #----------------------------------------------------------------------------

    pl.figure(figsize=(8,8))
    pl.contourf(xi1,xi2,lensed_image_d)
    pl.plot(lens_P['ximg'][np.nonzero(lens_P['ximg'])], lens_P['yimg'][np.nonzero(lens_P['yimg'])], 'bx')
    
    return 0


def cross_check_with_lensed_QSOs(lensID):
    # stack the lensed hosts and lensed points to verify the calculation
    # make some plots.
    return 0

In [16]:
if __name__ == '__main__':

    from tqdm import tqdm_notebook

    dsx = 0.01  # pixel size per side, arcseconds
    nnn = 1000  # number of pixels per side
    xi1, xi2 = ole.make_r_coor(nnn, dsx)

    hdulist = pyfits.open("./data/twinkles_lenses_v2.fits")
    shb, shd = load_in_data_sne()
    
    print shb.index
        
    for i in tqdm_notebook(shb.index[:10], desc="Main Loop"):
        lensP, srcPb, srcPd = create_cats_sne(i, hdulist, shb, shd)
        generate_lensed_host(xi1, xi2, lensP, srcPb, srcPd)

Int64Index([   0,    2,    4,    6,    8,   10,   12,   14,   18,   20,
            ...
            1979, 1981, 1983, 1987, 1989, 1991, 1993, 1995, 1999, 2001],
           dtype='int64', length=904)


HBox(children=(IntProgress(value=0, description=u'Main Loop', max=10), HTML(value=u'')))

1012
[  0   1   2   3   4   5   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 2

IndexError: index 0 is out of bounds for axis 0 with size 0